### ПРЕДОБРАБОТКА И ГЕНЕРАЦИЯ НОВЫХ ПРИЗНАКОВ

Необходимые импорты

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_SECRET_ACCESS_KEY")


In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, KBinsDiscretizer, PolynomialFeatures, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from autofeat import AutoFeatRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.catboost
import pandas as pd
from mlflow.models.signature import infer_signature
from catboost import Pool

In [5]:
df_path = "/home/mle-user/mle_projects/mle-project-sprint-2-v001/df_filtered.csv"
df = pd.read_csv(df_path)
df.head(5)

,id,flat_id,building_id,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,...,is_apartment,studio,total_area,price,decade,distance_from_moscow_center,district,okrug,okrug_num,log_price
0,2,1,18012,2001,2,55.794849,37.608013,3.00,97,10,...,False,False,43.000000,13500000.0,2000,4.623059,Марьина Роща,Северо-Восточный,0,16.418200
1,3,2,17821,2000,4,55.740040,37.761742,2.70,80,10,...,False,False,56.000000,13500000.0,2000,8.954583,Перово,Восточный,1,16.418200
2,4,3,18579,2002,4,55.672016,37.570877,2.64,771,17,...,False,False,76.000000,20000000.0,2000,9.627130,Черёмушки,Юго-Западный,2,16.811243
3,5,4,9293,1971,1,55.808807,37.707306,2.60,208,9,...,False,False,24.000000,5200000.0,1970,8.156988,Богородское,Восточный,1,15.464169
4,6,5,23964,2017,4,55.724728,37.743069,2.70,192,17,...,False,False,51.009998,8490104.0,2010,8.312821,Рязанский,Юго-Восточный,3,15.954412


In [6]:
# чек на всякий случай
df.dtypes

id                               int64
flat_id                          int64
building_id                      int64
build_year                       int64
building_type_int                int64
latitude                       float64
longitude                      float64
ceiling_height                 float64
flats_count                      int64
floors_total                     int64
has_elevator                      bool
floor                            int64
kitchen_area                   float64
living_area                    float64
rooms                            int64
is_apartment                      bool
studio                            bool
total_area                     float64
price                          float64
decade                           int64
distance_from_moscow_center    float64
district                        object
okrug                           object
okrug_num                        int64
log_price                      float64
dtype: object

### Скейлинг признаков

Перейдем к автогенерации признаков. Используем:
- PolynomialFeatures для колонок с комната, этажами и площадями
- KBinsDiscretizer для колонки расстоянием до центра Москвы
- OrdinalEncoder возьмем для колонки с десятилетием построения, т.к. нам надо, чтобы порядок (=новизна) учитывался моделью.
- OnehotEncoder не будем использовать ни для округа, ни для района, ни для типа здания. Мы их передадим в catboost как есть.

In [12]:
#  Колонки 
poly_cols = ['ceiling_height', 'flats_count', 'floors_total', 'floor',
             'kitchen_area', 'living_area', 'rooms', 'total_area']
kbins_cols = ['distance_from_moscow_center']
ordinal_cols = ['decade']
bool_cols = ['has_elevator', 'is_apartment', 'studio']

passthrough_cols = [
    'id', 'flat_id', 'building_id', 'build_year',
    'latitude', 'longitude',  'okrug_num', 'log_price', 'building_type_int', 'okrug',
    'district'
]
cat_features = ['building_type_int', 'okrug']
target_col = 'price'

# X и y 
X = df.drop(columns=[target_col])
y = df[target_col]

#  Сохраняем passthrough_cols отдельно 
passthrough_df = X[passthrough_cols].copy()

#  Остальные колонки для трансформаций 
X_to_transform = X.drop(columns=passthrough_cols)

# --- ColumnTransformer ---
preprocessor = ColumnTransformer(transformers=[
    ('poly', PolynomialFeatures(degree=2, include_bias=False), poly_cols),
    ('kbins', KBinsDiscretizer(n_bins=5, encode='onehot-dense', strategy='quantile'), kbins_cols),
    ('ordinal', OrdinalEncoder(), ordinal_cols),
    ('bool', 'passthrough', bool_cols)
], remainder='drop')

pipeline = Pipeline([('preprocessor', preprocessor)])

#  Фитим и трансформируем 
X_transformed = pipeline.fit_transform(X_to_transform, y)

#  Имена колонок 
try:
    transformed_names = pipeline.get_feature_names_out()
except:
    transformed_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

X_transformed_df = pd.DataFrame(X_transformed, columns=transformed_names, index=X.index)

print("Форма финального X_transformed_df:", X_transformed_df.shape)



Форма финального X_transformed_df: (127711, 53)


### Автогенерация признаков

Переходим к автогенерации признаков через AutoFeatRegressor. Чтобы не плодить 500 колонок, выберем самые распространенные трансформации: ['log', '1/', 'sqrt', куб, квадрат].

In [13]:
#  AutoFeatRegressor на числовых исходных колонках 
af_reg = AutoFeatRegressor(
    verbose=1,
    feateng_steps=1,
    transformations = ["1/", "log", "sqrt"] 

)

numeric_for_autofeat = poly_cols + kbins_cols
X_autofeat = af_reg.fit_transform(X[numeric_for_autofeat], y)
X_autofeat_new = X_autofeat.drop(columns=numeric_for_autofeat, errors='ignore')

#  Финальный датасет: passthrough + трансформированные + autofeat 
X_final = pd.concat([passthrough_df, X_transformed_df, X_autofeat_new], axis=1)

print("Форма финального X_final:", X_final.shape)
print("Колонки passthrough на месте:", all(c in X_final.columns for c in passthrough_cols))

2025-09-08 18:05:17,496 INFO: [AutoFeat] The 1 step feature engineering process could generate up to 27 features.
2025-09-08 18:05:17,497 INFO: [AutoFeat] With 127711 data points this new feature matrix would use about 0.01 gb of space.
2025-09-08 18:05:17,507 INFO: [feateng] Step 1: transformation of original features


2025-09-08 18:05:18,167 INFO: [feateng] Generated 27 transformed features from 9 original features - done.
2025-09-08 18:05:18,185 INFO: [feateng] Generated altogether 27 new features in 1 steps
2025-09-08 18:05:18,185 INFO: [feateng] Removing correlated features, as well as additions at the highest level
2025-09-08 18:05:18,228 INFO: [feateng] Generated a total of 11 additional features
2025-09-08 18:05:18,252 INFO: [featsel] Feature selection run 1/5


[featsel] Scaling data...done.


2025-09-08 18:05:21,038 INFO: [featsel] Feature selection run 2/5
2025-09-08 18:05:23,685 INFO: [featsel] Feature selection run 3/5
2025-09-08 18:05:26,530 INFO: [featsel] Feature selection run 4/5
2025-09-08 18:05:29,354 INFO: [featsel] Feature selection run 5/5
2025-09-08 18:05:32,082 INFO: [featsel] 17 features after 5 feature selection runs
2025-09-08 18:05:32,231 INFO: [featsel] 17 features after correlation filtering
2025-09-08 18:05:33,235 INFO: [featsel] 17 features after noise filtering
2025-09-08 18:05:33,236 INFO: [AutoFeat] Computing 8 new features.


2025-09-08 18:05:34,239 INFO: [AutoFeat]     8/    8 new features ...done.
2025-09-08 18:05:34,248 INFO: [AutoFeat] Final dataframe with 17 feature columns (8 new).
2025-09-08 18:05:34,249 INFO: [AutoFeat] Training final regression model.


2025-09-08 18:05:34,625 INFO: [AutoFeat] Trained model: largest coefficients:
2025-09-08 18:05:34,626 INFO: -28316844.265038334
2025-09-08 18:05:34,627 INFO: 137642063.637287 * 1/total_area
2025-09-08 18:05:34,628 INFO: 28958126.260046 * 1/distance_from_moscow_center
2025-09-08 18:05:34,629 INFO: 28413042.161550 * 1/living_area
2025-09-08 18:05:34,630 INFO: 27029558.545558 * 1/kitchen_area
2025-09-08 18:05:34,630 INFO: -8997670.810305 * 1/floors_total
2025-09-08 18:05:34,631 INFO: 5440678.178909 * ceiling_height
2025-09-08 18:05:34,632 INFO: 2923547.394878 * 1/flats_count
2025-09-08 18:05:34,633 INFO: -1755428.560856 * rooms
2025-09-08 18:05:34,634 INFO: -1683582.852103 * 1/floor
2025-09-08 18:05:34,635 INFO: 351617.201446 * total_area
2025-09-08 18:05:34,635 INFO: 305989.047430 * kitchen_area
2025-09-08 18:05:34,636 INFO: -191917.056979 * distance_from_moscow_center
2025-09-08 18:05:34,637 INFO: 170111.551161 * log(flats_count)
2025-09-08 18:05:34,637 INFO: 49399.104154 * living_area


Форма финального X_final: (127711, 72)
Колонки passthrough на месте: True


Сохраняем это для дальнейшего логирования

In [14]:
# Создаем папку для артефактов, если ее нет
ARTIFACTS_DIR = "feature_generation_artifacts"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Создаем копию X_final и добавляем таргет
X_final_with_target = X_final.copy()
X_final_with_target['price'] = y  # добавляем колонку таргета

# Путь к CSV файлу
X_FINAL_PATH = os.path.join(ARTIFACTS_DIR, "X_final.csv")

# Сохраняем DataFrame
X_final_with_target.to_csv(X_FINAL_PATH, index=False)

print(f"X_final с таргетом сохранен по пути: {X_FINAL_PATH}")


X_final с таргетом сохранен по пути: feature_generation_artifacts/X_final.csv


### Обучение и оценка качества модели

In [15]:
X_model = X_final.drop(columns=['id', 'flat_id', 'building_id', 'latitude', 'longitude', 'district','okrug_num', 'log_price', 'build_year'])

# Категориальные признаки для CatBoost
cat_features = ['okrug',  'building_type_int']

# Разбивка данных

X_train, X_test, y_train, y_test = train_test_split(
    X_model, y, test_size=0.2, random_state=42
)


# CatBoostRegressor 

model_params = {
    'depth': 3,
    'learning_rate': 0.01,
    'iterations': 1000,
    'l2_leaf_reg': 1,
    'eval_metric': 'RMSE',
    'random_seed': 42,
    'verbose': 100
}

catboost_model = CatBoostRegressor(**model_params)

catboost_model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    use_best_model=True,
    cat_features=cat_features
)

# Обучение и метрики

y_pred = catboost_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"R2: {r2:.3f}")

0:	learn: 11168949.4682682	test: 11165796.7136388	best: 11165796.7136388 (0)	total: 89.3ms	remaining: 1m 29s
100:	learn: 7034321.2025122	test: 7057363.9069943	best: 7057363.9069943 (100)	total: 3.65s	remaining: 32.5s
200:	learn: 5664866.0677018	test: 5714870.1252136	best: 5714870.1252136 (200)	total: 6.94s	remaining: 27.6s
300:	learn: 5168762.2081253	test: 5248830.7844430	best: 5248830.7844430 (300)	total: 10.4s	remaining: 24.2s
400:	learn: 4969287.4888107	test: 5064766.1052301	best: 5064766.1052301 (400)	total: 13.8s	remaining: 20.7s
500:	learn: 4864668.2747619	test: 4970465.1679202	best: 4970465.1679202 (500)	total: 17.3s	remaining: 17.2s
600:	learn: 4797530.6071316	test: 4910397.4908275	best: 4910397.4908275 (600)	total: 20.6s	remaining: 13.7s
700:	learn: 4752071.2889796	test: 4870212.6570290	best: 4870212.6570290 (700)	total: 23.9s	remaining: 10.2s
800:	learn: 4714881.3130560	test: 4836960.3003420	best: 4836960.3003420 (800)	total: 27.1s	remaining: 6.73s
900:	learn: 4684000.7921409

/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv-1/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


### Логирование

In [37]:

# Настройка MLflow 
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("feature_generation")

#  Пути к артефактам 
X_FINAL_PATH = "X_final.csv"
NOTEBOOK_PATH = "feature_generation.ipynb"


#  Категориальные признаки 
cat_features = ['okrug', 'district', 'building_type_int']

#  Для сигнатуры преобразуем категориальные колонки в строки 
X_sig = X_final.copy()
X_sig[cat_features] = X_sig[cat_features].astype(str)

#  Получение сигнатуры
signature = infer_signature(X_sig, catboost_model.predict(Pool(X_final, cat_features=cat_features)))

#  Логирование в MLflow 
with mlflow.start_run(run_name="feature_generation_run"):

    # 1. Параметры модели (если есть словарь с ними)
    mlflow.log_params(model_params)

    # 2. Метрики (если они уже посчитаны)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)

    # 3. Модель с сигнатурой и регистрация в Model Registry
    mlflow.catboost.log_model(
        catboost_model,
        artifact_path="catboost_model",
        signature=signature,
        registered_model_name="FeatureGen_CatBoost_Model"
    )

    # 4. Артефакты: датасет и ноутбук
    mlflow.log_artifact(X_FINAL_PATH, artifact_path="data")
    mlflow.log_artifact(NOTEBOOK_PATH, artifact_path="notebook")

print("Модель, сигнатура и артефакты успешно залогированы в MLflow!")

/home/mle-user/mle_projects/mle-project-sprint-2-v001/.venv-1/lib/python3.10/site-packages/mlflow/models/signature.py:212: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input) if model_input is not None else None
2025/09/07 23:03:14 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed. The model artifacts have be

Модель, сигнатура и артефакты успешно залогированы в MLflow!
